In [3]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

examples = [
    {
        "movie_title": "The Lion King",
        "answer": "🦁👑😭",
    },
    {
        "movie_title": "Pirates of the Caribbean",
        "answer": "🏴‍☠️🚣‍♂️💎",
    },
    {
        "movie_title": "Frozen",
        "answer": "❄️👸🎶",
    },
    {
        "movie_title": "Jurassic Park",
        "answer": "🦖🌴🏃‍♂️",
    },
    {
        "movie_title": "The Matrix",
        "answer": "🕶️💻🐇",
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "By this movie title {movie_title}, could you respond me the three characters of emoju that can represent this movie?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Response should be ONLY three emojis."),
        example_prompt,
        ("human", "By this movie title {movie_title}, could you respond me the three characters of emoju that can represent this movie?"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

def get_last_movie_title():
    history = memory.load_memory_variables({})
    if history:
        last_interaction = history[-1]
        movie_title = last_interaction.get('input', 'No movie title in memory.')
        return movie_title
    else:
        return "No memory of any movie titles."

def query_memory_for_movie():
    last_movie_title = get_last_movie_title()
    print(f"The last movie title asked about was: {last_movie_title}")



chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(movie_title):
    result = chain.invoke({"movie_title": movie_title})
    memory.save_context(
        {"input": movie_title},
        {"output": result.content},
    )
    print(result.content)

In [4]:
invoke_chain("The Godfather")

👨‍👧🔫🇮🇹


In [22]:
invoke_chain("Tenet")

⏳🔄🔫


In [ ]:
query_memory_for_movie()